In [9]:
from pathlib import Path

from openai import OpenAI
from pydantic import BaseModel

In [2]:
files = list(Path('../data').rglob('*.md'))

In [3]:
test = files[0]

In [4]:
with open(test, "r") as f:
    email = f.read()

In [7]:
with open("../data/reading_history.txt", "r") as f:
    reading_history = f.readlines()

In [10]:
class NewsletterItem(BaseModel):
    headline: str
    url: str
    description: str